In [2]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import json
import plotly.graph_objects as go

In [3]:
with open("power-bi-colors.json", "r") as f:
    colors = json.load(f)

In [3]:
data_dirs = [i for i in os.listdir("data") if "scraping" not in i]

In [4]:
dates = [pd.to_datetime(i.split("_")[1].split(" ")[0]) for i in data_dirs]

In [5]:
dfs = [pd.read_csv("data/" + i) for i in data_dirs]


for idx, df in enumerate(dfs):
    df["version"] = dates[idx]

dfs = pd.concat(dfs)

In [13]:
dfs.loc[dfs["price"] == "auf", "price"] = np.nan
dfs["price"] = dfs["price"].astype(np.float32)
dfs = dfs.drop_duplicates(subset=["id", "version"])

In [22]:
v1 = dfs.version.iloc[0]

In [32]:
ids = list(dfs.id)

In [57]:
def show_listing(data, id_:str) -> pd.DataFrame:

    layout = go.Layout({"plot_bgcolor":"white"})
    listing_data = data.loc[data.id == id_].sort_values("version")

    fig = px.line(listing_data, x="version", y="price", markers="point", color_discrete_sequence=colors["Classroom"])
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
    fig.update_layout(layout)
    return fig


In [55]:
price_change_id = []

for i in ids:
    if len(show_listing(dfs, i).price.unique()) != 1:
        price_change_id.append(i)

In [ ]:
show_listing(dfs, price_change_id[6])

In [8]:
price_by_day = dfs.groupby("version").agg(
    price_mean=("price", "mean"),
    n_listings=("id", "count")
)

In [ ]:
layout = go.Layout({"plot_bgcolor":"white"})
fig = px.line(data_frame=price_by_day, y="price_mean", color_discrete_sequence=colors["Classroom"])
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
fig.update_layout(layout)

In [ ]:
fig = px.bar(data_frame=price_by_day, y="n_listings", color_discrete_sequence=colors["City Park"])
fig.update_layout({"plot_bgcolor":"white"})
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')

In [6]:
data_full = pd.read_csv("data/4_scraping_results.csv", index_col=0)

In [7]:
data_full.area = data_full.area.str.replace(",", ".")
data_full.loc[data_full.area == "k.A.", "area"] = np.nan
data_full.area = data_full.area.astype(float)

In [8]:
data_full.loc[data_full.price == "auf", "price"] = np.nan
data_full.price = data_full.price.astype(float)

In [9]:
data_full.rooms = data_full.rooms.str.replace(",", ".")
data_full.loc[data_full.rooms == "k.A.", "rooms"] = np.nan
data_full.rooms = data_full.rooms.astype(float)

In [98]:
data_full.dropna(subset="price", inplace=True)

In [100]:
data_full.head(2)

,id,price,link,first_price,title,area,rooms,address,vermietet,offerer_address,...,wohnlage,Hausgeld,Baujahr,energieträger,effizienzklasse,bezug,lage,details,scraped_at,active
0,27tl55c,337620.0,https://www.immowelt.de/expose/27tl55c,337620,DACHGESCHOSS MIT ELBBLICK,66.20,3.0,01139 Dresden,False,"Westendstraße 3, 01187 Dresden",...,2. Geschoss (Dachgeschoss),NaN,NaN,NaN,NaN,NaN,"LageDie idyllische, ruhige Lage wird Sie begei...",ObjektbeschreibungIn ruhiger idyllischer Lage ...,True,NaN
1,277b95r,329500.0,https://www.immowelt.de/expose/277b95r,329500,3 Zimmer Wohnung mit Südausrichtung,74.03,3.0,Lili-Elbe-Straße 101307 Dresden,False,"Schützenplatz 3, 01067 Dresden",...,Erdgeschoss,NaN,2023.0,NaN,NaN,NaN,LageDas Haus Johanngarten entsteht auf der neu...,AusstattungDurch vier Ausstattungslinien - PUR...,True,NaN


In [103]:
data = data_full.drop(columns=["id", "first_price", "offerer_address", "link"])

In [108]:
data.wohnlage.unique()

array(['2. Geschoss (Dachgeschoss)', 'Erdgeschoss', '2. Geschoss',
       '1. Geschoss', 'Dachgeschoss', '3. Geschoss',
       '3. Geschoss (Dachgeschoss)', nan, '4. Geschoss',
       '4. Geschoss (Dachgeschoss)', 'Souterrain',
       '2. Geschoss (Erdgeschoss)', '1. Geschoss (Dachgeschoss)',
       '5. Geschoss (Dachgeschoss)', '5. Geschoss', '6. Geschoss',
       '1. Geschoss (Erdgeschoss)', '8. Geschoss', '7. Geschoss'],
      dtype=object)

In [111]:
data["dachgeschoss"] = data.wohnlage.str.contains("Dachgeschoss")

In [112]:
X = data[["area", "rooms", "vermietet", "Baujahr", "dachgeschoss"]]
y = np.log(data["price"])

In [113]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [114]:
px.histogram(y)

In [120]:
lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=50)
s_imp = SimpleImputer()

X = s_imp.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [116]:
params_rf = {
    "n_estimators":[50,100,150],
    "max_depth":[2,3,4,5],
    "min_samples_split":[2,5,10]
}

In [117]:
cv = GridSearchCV(
    estimator=rf,
    param_grid=params_rf
)

In [ ]:
cv.fit(X, y)

In [127]:
pd.DataFrame(cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.066130,0.004665,0.004573,0.000354,2,2,50,"{'max_depth': 2, 'min_samples_split': 2, 'n_es...",0.674942,0.735252,0.691072,0.697858,0.576149,0.675054,0.053259,29
1,0.126620,0.003650,0.007985,0.000338,2,2,100,"{'max_depth': 2, 'min_samples_split': 2, 'n_es...",0.674393,0.727660,0.688447,0.694175,0.590286,0.674992,0.045822,30
2,0.184471,0.005143,0.011518,0.000177,2,2,150,"{'max_depth': 2, 'min_samples_split': 2, 'n_es...",0.663786,0.730346,0.690304,0.695386,0.585700,0.673104,0.048570,34
3,0.061472,0.000277,0.004151,0.000020,2,5,50,"{'max_depth': 2, 'min_samples_split': 5, 'n_es...",0.671750,0.727978,0.691425,0.700009,0.582965,0.674825,0.049361,31
4,0.121271,0.000142,0.007998,0.000224,2,5,100,"{'max_depth': 2, 'min_samples_split': 5, 'n_es...",0.676130,0.724513,0.699427,0.696015,0.584910,0.676199,0.048167,28
5,0.181059,0.000515,0.011586,0.000197,2,5,150,"{'max_depth': 2, 'min_samples_split': 5, 'n_es...",0.668360,0.733138,0.689997,0.694490,0.586172,0.674431,0.048822,32
6,0.060731,0.000164,0.004618,0.000120,2,10,50,"{'max_depth': 2, 'min_samples_split': 10, 'n_e...",0.657116,0.724373,0.693393,0.704640,0.588807,0.673666,0.047738,33
7,0.120819,0.000727,0.007756,0.000185,2,10,100,"{'max_depth': 2, 'min_samples_split': 10, 'n_e...",0.650377,0.729640,0.692776,0.692664,0.597571,0.672606,0.045136,35
8,0.179984,0.000622,0.011371,0.000256,2,10,150,"{'max_depth': 2, 'min_samples_split': 10, 'n_e...",0.666816,0.732355,0.682571,0.695605,0.579357,0.671341,0.050831,36
9,0.063550,0.000181,0.004286,0.000241,3,2,50,"{'max_depth': 3, 'min_samples_split': 2, 'n_es...",0.752971,0.799864,0.804375,0.845890,0.780252,0.796671,0.030566,27


In [79]:
# results = []

# for _ in range(20):
    
#     rf.fit(X_train, y_train)
#     results.append(rf.score(X_test, y_test))

In [ ]:
px.box(results)

In [123]:
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.726745315942952

In [126]:
lr.coef_

array([ 0.01428268,  0.01669826, -0.20886829,  0.00193836, -0.05630841])

In [31]:
rf.score(X_train, y_train)

0.9192144570312615